In [1]:
! pip install openai-whisper
! pip install setuptools-rust
! pip install asr-evaluation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=9c7b542cec721e58da3ac321a4ca90f2e2400e5b9c9d057b22ecd871155f633e
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/vistaar-indic-asr/kathbath/kathbath/hindi'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
test = r"/kaggle/input/vistaar-indic-asr/kathbath/kathbath/hindi/test/audio"

In [4]:
data = pd.read_csv(r"/kaggle/input/vistaar-indic-asr/kathbath/kathbath/hindi/test/sb_manifest/test_unknown.csv")
data.head()

,ID,duration,wav,spk_id,wrd
0,8c226612-389b-44d6-94bf-b63acc9b62ca,5.758563,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,8c226612-389b-44d6-94bf-b63acc9b62ca,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...
1,68625976-f3b8-41c0-b13c-312a2ef5ad8c,4.644000,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,68625976-f3b8-41c0-b13c-312a2ef5ad8c,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति
2,c83df8f9-0260-4db1-9cff-c8af6d2972ab,5.758563,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,c83df8f9-0260-4db1-9cff-c8af6d2972ab,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...
3,bfe1ddf2-224b-4238-a95c-69f5c45df721,8.661062,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,bfe1ddf2-224b-4238-a95c-69f5c45df721,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...
4,ada13396-8962-4fe6-9f7c-c9a4f18ebaba,4.992312,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,ada13396-8962-4fe6-9f7c-c9a4f18ebaba,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...


In [5]:
data.drop(columns={"ID","duration","spk_id"}, inplace=True)


In [6]:
data.head()

,wav,wrd
0,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...
1,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति
2,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...
3,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...
4,/nlsasfs/home/ai4bharat/ai4bharat-pr/speechtea...,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...


In [7]:
data["wav"][0]

'/nlsasfs/home/ai4bharat/ai4bharat-pr/speechteam/asr_datasets/kathbath/hindi/test/audio/1202/844424932919645-1202-f.wav'

In [8]:
old_path = '/nlsasfs/home/ai4bharat/ai4bharat-pr/speechteam/asr_datasets/kathbath/hindi/test/'
new_path = '/kaggle/input/vistaar-indic-asr/kathbath/kathbath/hindi/test/'


# Replace the old path with the new path in the 'wav' column
data['wav'] = data['wav'].str.replace(old_path, new_path)

In [9]:
data.head()

,wav,wrd
0,/kaggle/input/vistaar-indic-asr/kathbath/kathb...,सुधीर मिश्रा की ये फिल्म लेखक शरत चंद्र चट्टोप...
1,/kaggle/input/vistaar-indic-asr/kathbath/kathb...,खोज कम मांग के लिए इंजन में वेबसाइट पदोन्नति
2,/kaggle/input/vistaar-indic-asr/kathbath/kathb...,ट्रेन लक्ष्मीबाई नगर भी नहीं पहुंचती कि लोगों ...
3,/kaggle/input/vistaar-indic-asr/kathbath/kathb...,नई पॉलिसी के तहत एक्सट्रा चार्ज देकर ज्यादा एफ...
4,/kaggle/input/vistaar-indic-asr/kathbath/kathb...,उन्होंने वन मुख्यालय से प्रस्ताव पर अनुमति प्र...


In [10]:
print(len(data["wav"]))


1929


In [11]:
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:09<00:00, 160MiB/s]


In [13]:
base = r"/kaggle/input/vistaar-indic-asr/kathbath/kathbath/hindi/test/audio/"
def process_audio(audio):
    audio = whisper.pad_or_trim(whisper.load_audio(audio))
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions(fp16=False, beam_size=5, without_timestamps=True, language='hi')
    res = whisper.decode(model, mel, options)
    return res.text

In [14]:
from tqdm import tqdm

In [17]:
transcriptions = []

for i in tqdm(range(len(data))):
    a = data["wav"][i]
    r = process_audio(a)
    transcriptions.append(r)
    

100%|██████████| 1929/1929 [1:42:00<00:00,  3.17s/it]


In [18]:
print(transcriptions)

['सुदिर मिश्रा की ये फिर्म लेकिक शर्थ चंटर चटो पाध्या के पन्यास देविडास पर आधारित होगी', 'खोज कम मांग के लिए इंजन में वेबसाइट पदोनती', 'ट्रेन लक्षमीबाई नगर भी नहीं पोंषती कि लोगो का बत्याना बंद हो जाता है', 'नहीं पालिसी के तहत एक्स्टर चार्ज देकर जादा एफायर भी लिया जा सकता है', 'वोनोंने वन मुख्याले से प्रस्ताव पर अनुमती प्रदान करने का आग्रे किया है', 'उश्या ने ओटो चालक को बताया की उसे बिंड जाना है', 'देश की जन्ता अपने आप अन्ना की टीम के सवालो का जवाब जुनावों में देगी', 'शाशण दौर मंजुरी मिलते ही ओपन जेल का निर्मानी कार्णी शुरू हो गया।', 'इसके अलावा गुजरात के एहमदा बाद में आज जगर्नात यात्रा निकाली जाएगी', 'अबी या साफ नहीं है कि किरण नहीं पार्टी बनाएंगे अत्वा नहीं', 'लोग रो रहे थे तो कोई उन्हें धांडस मदा रहा था', 'स्वतन्तिता सैनानी अमर्नात डोग्रा को पॉलिस के जवानों ने उन्हें अन्तिम विदाई दी', 'उन फोटोस में कपेलीक प्यारी सी बच्छी को थामे होय थे', 'होली के 8 देन बाद होती है शीत ला माता की पुजा', 'आस्था केंद्र इस मंदिर में लोग प्रतिदिन पूजा करते हैं', 'दोनी ने मैदान पर आते ही आक्रामक अन्दास म

In [19]:
data["transcriptions"] = transcriptions
data.to_csv('results-kathbath-whisper.csv',index=False)

In [20]:
references = list(data["wrd"])

In [21]:
print(len(references))
print(len(transcriptions))

1929
1929


In [22]:
! pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.0 MB/s eta 0:00:00a 0:00:01


In [23]:
import jiwer

In [24]:
wer = jiwer.wer(references,transcriptions)

print(f"WER: {wer * 100:.2f} %")

WER: 40.75 %
